# <span style="color:#ff1414"> BEDtools analysis. </span>

This is a script to answer research questions outlined elsewhere. In summary, this script:

1. compares methylation results between different methylation-callers, and between different methylation sequencing methods.

2. compares methylation between genes and non-gene regions

3. compares methylation between transposons and non-repetitive regions

4. compares transposons and genes


Note:
- PB/pb = PacBio
- ONT/ont = Oxford Nanopore Technology
- NP = Nanopolish

In [4]:
import pybedtools
import scipy

import matplotlib.patches as mpatches

import numpy as np # need for  stats

from scipy.stats import wilcoxon
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

In [39]:
# load modules
import os
import glob
import pprint
from pybedtools import BedTool
import pandas as pd
from scipy.stats import spearmanr

In [ ]:
DIRS['BASE1'] = '/home/anjuni/methylation_calling/pacbio'

DIRS['BED_INPUT'] = os.path.join(DIRS['BASE2'], 'bedtools_output', 'sequencing_comparison')

DIRS['FIGURES'] = os.path.join(DIRS['BASE2'], 'figures')

DIRS['I_FROM_C'] = os.path.join(DIRS['BASE2'], 'bedtools_output', 'intersects_from_cutoffs')
DIRS['BED_CUTOFFS'] = os.path.join(DIRS['BASE1'], 'input', 'sorted_bed_files', 'cutoffs')
DIRS['6MA_CUTOFFS'] = os.path.join(DIRS['BASE1'], 'input', 'sorted_bed_files', 'cutoffs_6mA')
DIRS['5MC_CUTOFFS'] = os.path.join(DIRS['BASE1'], 'input', 'sorted_bed_files', 'cutoffs_5mC')

In [208]:
#First we need to define the base dirs
DIRS = {}
DIRS['BASE2'] = '/home/anjuni/analysis'
DIRS['FEATURES'] = os.path.join(DIRS['BASE2'], 'coverage', 'feature_files')
DIRS['RAND'] = os.path.join(DIRS['BASE2'], 'coverage', 'randomisation')
DIRS['WINDOW_OUTPUT'] = os.path.join(DIRS['BASE2'], 'windows')
DIRS['WINDOW_INPUT'] = os.path.join(DIRS['BASE2'], 'input_for_windows')
DIRS['REF'] = '/home/anjuni/Pst_104_v13_assembly/'
DIRS['COVERAGE'] = os.path.join(DIRS['BASE2'], 'coverage')
DIRS['GFF_INPUT'] = os.path.join(DIRS['BASE2'], 'gff_output')
DIRS['TE_SF'] = os.path.join(DIRS['COVERAGE'], 'superfamily_files')

In [21]:
#Quick chech if directories exist
for value in DIRS.values():
    if not os.path.exists(value):
        print('%s does not exist' % value)

In [79]:
#Make filepaths
bed_file_list = [fn for fn in glob.iglob('%s/*.bed' % DIRS['BED_INPUT'], recursive=True)]
gff_file_list = [fn for fn in glob.iglob('%s/*anno.gff3' % DIRS['GFF_INPUT'], recursive=True)]
te_file_list = [fn for fn in glob.iglob('%s/*.gff' % DIRS['GFF_INPUT'], recursive=True)]

In [80]:
#Check that the list works
print(*bed_file_list, sep='\n')
print(*gff_file_list, sep='\n')
print(*te_file_list, sep='\n')

/home/anjuni/analysis/bedtools_output/sequencing_comparison/5mC_plus_tombo_sorted.bed
/home/anjuni/analysis/bedtools_output/sequencing_comparison/5mC_CpG_tombo_np.bed
/home/anjuni/analysis/bedtools_output/sequencing_comparison/5mC_tombo_np.bed
/home/anjuni/analysis/bedtools_output/sequencing_comparison/5mC_plus_CpG_np_tombo.bed
/home/anjuni/analysis/bedtools_output/sequencing_comparison/5mC_hc_tombo_sorted.CpG.plus.bed
/home/anjuni/analysis/bedtools_output/sequencing_comparison/5mC_s_nanopolish.bed
/home/anjuni/analysis/bedtools_output/sequencing_comparison/nanopolish_rerun_subtract.bed
/home/anjuni/analysis/bedtools_output/sequencing_comparison/5mC_hc_nanopolish_sorted.bed
/home/anjuni/analysis/bedtools_output/sequencing_comparison/5mC_plus_CpG_tombo_np.bed
/home/anjuni/analysis/bedtools_output/sequencing_comparison/nano_plus_tombo_overlap.bed
/home/anjuni/analysis/bedtools_output/sequencing_comparison/5mC_tombo_sorted.bed
/home/anjuni/analysis/bedtools_output/sequencing_comparison/5m

## <span style='color:#144fff'> 4. Making windows. <span/>

In [22]:
# Make folder for windows. Each BED file will contain a series of windows
#os.mkdir(DIRS['WINDOW_OUTPUT'])
#os.mkdir()
# Define inputs
gene_fn = '/home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg.anno.sorted.gff3'
lt_gene_fn = '/home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg.TE.sorted.gff3'
reference_genome = os.path.join(DIRS['REF'], 'Pst_104E_v13_ph_ctg.fa')

In [58]:
# Make the genome size file for windows
!samtools faidx /home/anjuni/Pst_104_v13_assembly/Pst_104E_v13_ph_ctg.fa
!cut -f 1,2 /home/anjuni/Pst_104_v13_assembly/Pst_104E_v13_ph_ctg.fa.fai > /home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg.genome_file
# Note: this does put the p contig values before h contig ones, while annotation files put h contig before p contig
# May be a problem in the future but probs not
# Sorted it anyway below, as reference genome fasta had contigs in that order arbitrarily:
!/home/anjuni/myapps/gff3sort/gff3sort.pl /home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg.genome_file >  /home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg.sorted.genome_file

Smartmatch is experimental at /home/anjuni/myapps/gff3sort/gff3sort.pl line 68.
Use of uninitialized value $pos in hash element at /home/anjuni/myapps/gff3sort/gff3sort.pl line 67, <$_[...]> line 1.
Use of uninitialized value $pos in hash element at /home/anjuni/myapps/gff3sort/gff3sort.pl line 67, <$_[...]> line 2.
Use of uninitialized value $pos in hash element at /home/anjuni/myapps/gff3sort/gff3sort.pl line 67, <$_[...]> line 3.
Use of uninitialized value $pos in hash element at /home/anjuni/myapps/gff3sort/gff3sort.pl line 67, <$_[...]> line 4.
Use of uninitialized value $pos in hash element at /home/anjuni/myapps/gff3sort/gff3sort.pl line 67, <$_[...]> line 5.
Use of uninitialized value $pos in hash element at /home/anjuni/myapps/gff3sort/gff3sort.pl line 67, <$_[...]> line 6.
Use of uninitialized value $pos in hash element at /home/anjuni/myapps/gff3sort/gff3sort.pl line 67, <$_[...]> line 7.
Use of uninitialized value $pos in hash element at /home/anjuni/myapps/gff3sort/gff3sor

In [7]:
# Define all file paths for window BED files
window_fn_dict = {}
window_bed_dict = {}
#window_fn_dict['100kb'] = os.path.join(DIRS['WINDOW_OUTPUT'], 'Pst_104E_v13_ph_ctg_w100kb.bed')
#window_fn_dict['30kb'] = os.path.join(DIRS['WINDOW_OUTPUT'], 'Pst_104E_v13_ph_ctg_w30kb.bed')
window_fn_dict['10kb'] = os.path.join(DIRS['WINDOW_OUTPUT'], 'Pst_104E_v13_ph_ctg_w10kb.bed')
#window_fn_dict['100kb_s20kb'] = os.path.join(DIRS['WINDOW_OUTPUT'], 'Pst_104E_v13_ph_ctg_w100kb_s20kb.bed')
#window_fn_dict['30kb_s6kb'] = os.path.join(DIRS['WINDOW_OUTPUT'], 'Pst_104E_v13_ph_ctg_w30kb_s6kb.bed')
window_fn_dict['10kb_s2kb'] = os.path.join(DIRS['WINDOW_OUTPUT'], 'Pst_104E_v13_ph_ctg_w10kb_s2kb.bed')
genome_size_f_fn = os.path.join(DIRS['WINDOW_INPUT'], 'Pst_104E_v13_ph_ctg.sorted.genome_file')

In [23]:
# Check whether the dictionary looks nice :) (it does!) :D
pprint.pprint(window_fn_dict)

{'10kb': '/home/anjuni/analysis/windows/Pst_104E_v13_ph_ctg_w10kb.bed',
 '10kb_s2kb': '/home/anjuni/analysis/windows/Pst_104E_v13_ph_ctg_w10kb_s2kb.bed'}


In [73]:
# Make the actual windows! :D
#!bedtools makewindows -g {genome_size_f_fn} -w 100000 > {window_fn_dict['100kb']}
#!bedtools makewindows -g {genome_size_f_fn} -w 30000 > {window_fn_dict['30kb']}
!bedtools makewindows -g {genome_size_f_fn} -w 10000 > {window_fn_dict['10kb']}

In [10]:
# Make sliding windows
#!bedtools makewindows -g {genome_size_f_fn} -w 100000 -s 20000 > {window_fn_dict['100kb_s20kb']}
#!bedtools makewindows -g {genome_size_f_fn} -w 30000 -s 6000 > {window_fn_dict['30kb_s6kb']}
!bedtools makewindows -g {genome_size_f_fn} -w 10000 -s 2000 > {window_fn_dict['10kb_s2kb']}

In [1]:
# Function to easily convert the values in the file name dict into bedtools objects
def make_bed_dict(fn_dict):
    """Takes an input filename dictionary and outputs a dictionary of pybedtools objects for the filenames."""
    bed_dict = {}
    for key, value in fn_dict.items():      
        bed_dict[key] = BedTool(value)
    return bed_dict

In [15]:
#new make a bedtools window dataframe
window_bed_dict = make_bed_dict(window_fn_dict)

In [16]:
# Check whether the bed file dictionary looks nice :) (it does!) :D
pprint.pprint(window_bed_dict)

{'10kb': <BedTool(/home/anjuni/analysis/windows/Pst_104E_v13_ph_ctg_w10kb.bed)>,
 '10kb_s2kb': <BedTool(/home/anjuni/analysis/windows/Pst_104E_v13_ph_ctg_w10kb_s2kb.bed)>}


## <span style='color:#148aff'> 5. Coverage analysis of methylation with gene annotation files. <span/>

# <span style='color:red'> Move this to anno file prep notebook. <span/>

### <span style='color:#148aff'> 5.A Make feature files of effector proteins, non-effectors, genes, BUSCO genes and TE superfamilies. <span/>

In [17]:
%%bash
# Downloading the effector file (need raw version with only the file)
cd /home/anjuni/analysis/input_for_windows
wget https://raw.githubusercontent.com/BenjaminSchwessinger/Pst_104_E137_A-_genome/master/supplemental_files/Supplemental_file_9.txt
mv Supplemental_file_9.txt Candidate_effectors.txt

--2018-08-07 15:30:17--  https://raw.githubusercontent.com/BenjaminSchwessinger/Pst_104_E137_A-_genome/master/supplemental_files/Supplemental_file_9.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.80.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.80.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49295 (48K) [text/plain]
Saving to: 'Supplemental_file_9.txt’

     0K .......... .......... .......... .......... ........  100% 2.80M=0.02s

2018-08-07 15:30:22 (2.80 MB/s) - 'Supplemental_file_9.txt’ saved [49295/49295]



In [24]:
# Make a GFF file of effector proteins

# First extract all lines with genes (not exon or CDS) from the gene annotation file
! grep 'gene' /home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg.anno.sorted.gff3 > /home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg.anno.sorted.genes_only.gff3

In [17]:
# Then extract the effector lines from the gene file

# First write grep function for python
import fileinput
import re
import glob

def grep(PAT, FILES):
    """Same function as 'grep' in bash."""
    fileinput.close() # close the file in case the iterable was previously open to prevent the "input() already active" error
    for line in fileinput.input(glob.glob(FILES)):
        if re.search(PAT, line):
            #print(fileinput.filename(), fileinput.lineno(), line)
            return line
    fileinput.close()

# Write function for filtering effectors from gene file using grep            
def make_effector_gff(effector_list, gene_gff, out_gff):
    """Get effector protein features out of gene annotation files."""
    with open(out_gff, mode = 'w') as out_file:
        for effector in effector_list:
            print(grep(effector, gene_gff), file = out_file)
        
# And make a list of effectors
def make_effector_list(input_file):
    list_name = []
    with open(input_file) as file:
        for line in file:
            line = line.strip()
            list_name.append(line)
    return list_name

In [18]:
# Make the effector list
effectors = make_effector_list('/home/anjuni/analysis/input_for_windows/Candidate_effectors.txt')

In [19]:
# Check if list works (it does, but is not sorted)
print(effectors)

['evm.TU.pcontig_058.72', 'evm.TU.pcontig_017.202', 'evm.TU.pcontig_010.332', 'evm.TU.pcontig_010.240', 'evm.TU.pcontig_031.123', 'evm.TU.pcontig_008.86', 'evm.TU.pcontig_050.77', 'evm.TU.pcontig_016.165', 'evm.TU.pcontig_037.43', 'evm.TU.pcontig_007.151', 'evm.TU.pcontig_030.240', 'evm.TU.pcontig_078.71', 'evm.TU.pcontig_052.152', 'evm.TU.pcontig_157.8', 'evm.TU.pcontig_037.109', 'evm.TU.pcontig_005.418', 'evm.TU.pcontig_062.104', 'evm.TU.pcontig_010.352', 'evm.TU.pcontig_013.247', 'evm.TU.pcontig_064.41', 'evm.TU.pcontig_062.26', 'evm.TU.pcontig_026.251', 'evm.TU.pcontig_005.10', 'evm.TU.pcontig_013.152', 'evm.TU.pcontig_036.154', 'evm.TU.pcontig_003.543', 'evm.TU.pcontig_004.284', 'evm.TU.pcontig_005.98', 'evm.TU.pcontig_004.435', 'evm.TU.pcontig_002.333', 'evm.TU.pcontig_000.679', 'evm.TU.pcontig_011.263', 'evm.TU.pcontig_051.51', 'evm.TU.pcontig_004.18', 'evm.TU.pcontig_017.200', 'evm.TU.pcontig_011.238', 'evm.TU.pcontig_026.59', 'evm.TU.pcontig_165.8', 'evm.TU.pcontig_010.220', '

In [20]:
# Sort effector list
effectors.sort()
print(effectors) # Check if sorting worked (it did)

['evm.TU.hcontig_000_003.1', 'evm.TU.hcontig_000_003.10', 'evm.TU.hcontig_000_003.120', 'evm.TU.hcontig_000_003.158', 'evm.TU.hcontig_000_003.2', 'evm.TU.hcontig_000_003.20', 'evm.TU.hcontig_000_003.26', 'evm.TU.hcontig_000_003.314', 'evm.TU.hcontig_000_003.340', 'evm.TU.hcontig_000_003.380', 'evm.TU.hcontig_000_003.402', 'evm.TU.hcontig_000_003.419', 'evm.TU.hcontig_000_003.421', 'evm.TU.hcontig_000_003.423', 'evm.TU.hcontig_000_003.444', 'evm.TU.hcontig_000_003.450', 'evm.TU.hcontig_000_003.90', 'evm.TU.hcontig_000_031.4', 'evm.TU.hcontig_000_050.114', 'evm.TU.hcontig_000_050.122', 'evm.TU.hcontig_000_050.141', 'evm.TU.hcontig_000_050.144', 'evm.TU.hcontig_000_050.149', 'evm.TU.hcontig_000_050.34', 'evm.TU.hcontig_000_050.85', 'evm.TU.hcontig_000_050.87', 'evm.TU.hcontig_000_050.9', 'evm.TU.hcontig_000_050.93', 'evm.TU.hcontig_000_054.13', 'evm.TU.hcontig_000_054.43', 'evm.TU.hcontig_000_054.68', 'evm.TU.hcontig_001_001.103', 'evm.TU.hcontig_001_001.128', 'evm.TU.hcontig_001_001.129'

In [276]:
# Run the function to make a file of effectors
genes_only_fn = '/home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg.anno.sorted.genes_only.gff3'
effector_fn  = '/home/anjuni/analysis/input_for_windows/Pst_104E_v13_ph_ctg.effectors.gff3'
make_effector_gff(effectors, genes_only_fn, effector_fn)

In [846]:
# Move the effector file and all gene annotation files to the feature file directory
!cp /home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg.anno.sorted.genes_only.gff3 /home/anjuni/analysis/coverage/feature_files

In [849]:
%%bash
# Make a file of non-effector genes only
cd /home/anjuni/analysis/coverage/feature_files
subtractBed -a Pst_104E_v13_ph_ctg.anno.sorted.genes_only.gff3 -b Pst_104E_v13_ph_ctg.effectors.gff3 > Pst_104E_v13_ph_ctg.non_effectors.gff3

In [24]:
# Make a GFF file of LT genes

# First extract all lines with genes (not exon or CDS) from the gene annotation file
! grep 'gene' /home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg.TE.sorted.gff3 > /home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg.TE.sorted.genes_only.gff3

In [25]:
# Move the LT gene_only file to the feature file directory
!cp /home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg.TE.sorted.genes_only.gff3 /home/anjuni/analysis/coverage/feature_files

In [34]:
# Make a GFF file of combined genes

# First extract all lines with genes (not exon or CDS) from the gene annotation file
! grep 'gene' /home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg_combined_sorted_anno.gff3 > /home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg_combined_sorted_anno.genes_only.gff3

In [35]:
# Move the abogene_only file to the feature file directory
!cp /home/anjuni/analysis/gff_output/Pst_104E_v13_ph_ctg_combined_sorted_anno.genes_only.gff3 /home/anjuni/analysis/coverage/feature_files

# <span style='color:red'> Move this to anno file prep notebook. <span/>

### BUSCO files notes

- were made in the annotation prep notebook
- have gene ID instead of gene Name in the 'feature' column, so I will make a new column with gene Name, using the original annotation file, and replace the gene ID with gene Name

In [154]:
#Inputs
all_genes_h = os.path.join(DIRS['GFF_INPUT'], 'Pst_104E_v13_h_ctg_combined_sorted_anno.genes_only.gff3')
all_genes_p = os.path.join(DIRS['GFF_INPUT'], 'Pst_104E_v13_p_ctg_combined_sorted_anno.genes_only.gff3')
busco_h = os.path.join(DIRS['GFF_INPUT'], 'busco', 'Pst_104E_v12_h_busco.gene.bed')
busco_p = os.path.join(DIRS['GFF_INPUT'], 'busco', 'Pst_104E_v12_p_busco.gene.bed')
non_busco_h = os.path.join(DIRS['GFF_INPUT'], 'busco', 'Pst_104E_v12_h_non_busco.gene.bed')
non_busco_p = os.path.join(DIRS['GFF_INPUT'], 'busco', 'Pst_104E_v12_p_non_busco.gene.bed')

In [111]:
def make_busco_list(busco_file):
    busco_gene_list = []
    df = pd.read_csv(busco_file, header=None, sep='\t')
    for index, row in df.iterrows():
        busco_gene_list.append(row[3])
    return busco_gene_list

In [112]:
busco_h_list = make_busco_list(busco_h)
busco_p_list = make_busco_list(busco_p)
non_busco_h_list = make_busco_list(non_busco_h)
non_busco_p_list = make_busco_list(non_busco_p)

In [187]:
gene_anno_fn = os.path.join(DIRS['GFF_INPUT'], 'Pst_104E_v13_ph_ctg_combined_sorted_anno.genes_only.gff3')

In [184]:
# write a function to make a list of gene Names
def get_busco_names(busco_gene_list, anno_fn):
    "This function goes through lines a gene annotation file and checks whether the gene_ID in the input list is in the line, and saves the corresponding gene name to a list."
    gene_name_list = []
    for gene in busco_gene_list:
        with open(anno_fn) as infile:
            for line in infile:
                line = line.rstrip()
                e = line.split('\t')
                if e[2] == 'gene':
                    f = e[-1].split(';')    
                    for i in f:
                        if i.startswith('ID=') and i.endswith(gene):
                            for i in f:
                                if i.startswith('Name='):
                                    name = i[5:]
                                    gene_name_list.append(name)
    return gene_name_list

In [188]:
busco_gene_h = get_busco_names(busco_h_list, gene_anno_fn)

In [189]:
busco_gene_h

['gene_model_hcontig_0000_03.12',
 'gene_model_hcontig_0000_03.19',
 'gene_model_hcontig_0000_03.29',
 'gene_model_hcontig_0000_03.83',
 'gene_model_hcontig_0000_03.87',
 'gene_model_hcontig_0000_03.91',
 'gene_model_hcontig_0000_03.126',
 'gene_model_hcontig_0000_03.174',
 'gene_model_hcontig_0000_03.180',
 'gene_model_hcontig_0000_03.192',
 'gene_model_hcontig_0000_03.215',
 'gene_model_hcontig_0000_03.224',
 'gene_model_hcontig_0000_03.230',
 'gene_model_hcontig_0000_03.232',
 'gene_model_hcontig_0000_03.244',
 'gene_model_hcontig_0000_03.246',
 'gene_model_hcontig_0000_03.254',
 'gene_model_hcontig_0000_03.269',
 'gene_model_hcontig_0000_03.272',
 'gene_model_hcontig_0000_03.275',
 'gene_model_hcontig_0000_03.326',
 'gene_model_hcontig_0000_03.328',
 'gene_model_hcontig_0000_03.336',
 'gene_model_hcontig_0000_03.343',
 'gene_model_hcontig_0000_03.371',
 'gene_model_hcontig_0000_03.375',
 'gene_model_hcontig_0000_03.381',
 'gene_model_hcontig_0000_03.403',
 'gene_model_hcontig_0000_

In [ ]:
#busco_gene_h = get_busco_names(busco_h_list, gene_anno_fn)
busco_gene_p = get_busco_names(busco_p_list, gene_anno_fn)
non_busco_gene_h = get_busco_names(non_busco_h_list, gene_anno_fn)
non_busco_gene_p = get_busco_names(non_busco_p_list, gene_anno_fn)

In [206]:
busco_gene_p

['gene_model_pcontig_000.11',
 'gene_model_pcontig_000.18',
 'gene_model_pcontig_000.22',
 'gene_model_pcontig_000.78',
 'gene_model_pcontig_000.82',
 'gene_model_pcontig_000.86',
 'gene_model_pcontig_000.119',
 'gene_model_pcontig_000.160',
 'gene_model_pcontig_000.166',
 'gene_model_pcontig_000.178',
 'gene_model_pcontig_000.204',
 'gene_model_pcontig_000.214',
 'gene_model_pcontig_000.220',
 'gene_model_pcontig_000.222',
 'gene_model_pcontig_000.235',
 'gene_model_pcontig_000.254',
 'gene_model_pcontig_000.269',
 'gene_model_pcontig_000.272',
 'gene_model_pcontig_000.275',
 'gene_model_pcontig_000.317',
 'gene_model_pcontig_000.319',
 'gene_model_pcontig_000.328',
 'gene_model_pcontig_000.336',
 'gene_model_pcontig_000.371',
 'gene_model_pcontig_000.376',
 'gene_model_pcontig_000.382',
 'gene_model_pcontig_000.407',
 'gene_model_pcontig_000.417',
 'gene_model_pcontig_000.424',
 'gene_model_pcontig_000.447',
 'gene_model_pcontig_000.449',
 'gene_model_pcontig_000.452',
 'gene_model_p

In [ ]:
non_busco_gene_p = get_busco_names(non_busco_p_list, gene_anno_fn)

In [190]:
print(len(busco_gene_h))

1293


In [136]:
print(len(busco_gene_h))
print(len(busco_gene_p))
print(len(non_busco_gene_h))
print(len(non_busco_gene_p))

1293
1444
13028
13028


In [203]:
# function to edit busco bed file and replace gene ID column with gene Name
def replace_id_with_name(in_fn, gene_list):
    "This function takes a busco bed file with gene ID's and a list of their corresponding gene Names, and makes a new busco bed file with gene Name instead of ID."
    df = pd.read_csv(in_fn, header=None, names=busco_headings, sep='\t')
    df['new_name'] = pd.Series(gene_list)
    df = df [['contig', 'start', 'stop', 'new_name', 'score', 'strand']]
    out_fn = in_fn.replace('gene', 'gene_name')
    df.to_csv(out_fn, header=None, index=None, sep='\t')

In [ ]:
replace_id_with_name(busco_h, busco_gene_h)

In [204]:
replace_id_with_name(busco_p, busco_gene_p)
replace_id_with_name(non_busco_h, non_busco_gene_h)
replace_id_with_name(non_busco_p, non_busco_gene_p)

In [196]:
df = pd.read_csv(in_fn, header=None, names=busco_headings, sep='\t')
df['new_name'] = pd.Series(busco_gene_h)
df = df [['contig', 'start', 'stop', 'new_name', 'score', 'strand']]
df.to_csv(out_fn, header=None, index=None, sep='\t')

In [195]:
df.head()

,contig,start,stop,new_name,score,strand
0,hcontig_000_003,45862,48106,gene_model_hcontig_0000_03.12,.,+
1,hcontig_000_003,65507,70053,gene_model_hcontig_0000_03.19,.,+
2,hcontig_000_003,131017,131797,gene_model_hcontig_0000_03.29,.,+
3,hcontig_000_003,341979,342986,gene_model_hcontig_0000_03.83,.,+
4,hcontig_000_003,361950,366382,gene_model_hcontig_0000_03.87,.,-


In [120]:
gene_name_list

['gene_model_hcontig_0000_03.12']

### TE superfamily files notes

- files present for p, h and combined ph contigs
- sf = superfamily

In [1270]:
# Read in the superfamily file as a pandas dataframe
te_sf_fn_p = os.path.join(DIRS['BASE2'], 'gff_output', 'Pst_104E_v13_p_ctg.REPET.sorted.filtered.superfamily.gff')
te_sf_fn_ph = os.path.join(DIRS['BASE2'], 'gff_output', 'Pst_104E_v13_p_ctg.REPET.sorted.filtered.superfamily.gff')
te_sf_fn_h = os.path.join(DIRS['BASE2'], 'gff_output', 'Pst_104E_v13_p_ctg.REPET.sorted.filtered.superfamily.gff')
te_sf_names = ['Contig', 'Source', 'Type', 'Start', 'Stop', 'Value', 'Strand', 'Space', 'Superfamily']

te_sf_df_p = pd.read_csv(te_sf_fn_p, sep='\t', header = None, names = te_sf_names)
te_sf_df_ph = pd.read_csv(te_sf_fn_ph, sep='\t', header = None, names = te_sf_names)
te_sf_df_h = pd.read_csv(te_sf_fn_h, sep='\t', header = None, names = te_sf_names)

In [ ]:
# write a function that loops over the last column in the data frame to make a list

In [1388]:
#Test to make a list of all sf
def make_superfamily_list(df):
    """Goes through dataframe and makes a list of all unique TE superfamily names."""
    superfamilies = []
    for sf in df['Superfamily']:
        if sf not in superfamilies:
            superfamilies.append(sf)
    return superfamilies

In [1391]:
te_sf_list_ph = make_superfamily_list(te_sf_df_ph)

In [1399]:
# write a function to loop over sf list and make a gff file for each superfamily
# these gff files will be used for window coverage analysis later
def filter_superfamilies(df, sf_list):
    """Filter out the TE's by their superfamily into separate files."""
    sf_dict = {}
    for sf in sf_list:
        filter_list = [sf]
        filtered_df = df[df['Superfamily'].isin(filter_list)]
        sf_dict[sf] = filtered_df
    return sf_dict

In [1400]:
te_sf_dict_ph = filter_superfamilies(te_sf_df_ph, te_sf_list_ph)

In [1415]:
# save out the dataframes in the dictionary as gff files
for key, value in te_sf_dict_ph.items():
    fn = 'Pst_104E_v13_ph_ctg.' + key + '.gff3'
    out_fn = os.path.join(DIRS['COVERAGE'], 'superfamily_files', fn)
    te_sf_dict_ph[key].to_csv(out_fn, sep='\t', header=None, index=None)

### <span style='color:#148aff'> 5.B Making dicts of feature file of effector proteins, non-effectors, genes, LT gene, BUSCO and TE superfamilies. <span/>

In [60]:
# Function that gives a dictionary of each input filehandle
def file_name_dict(file_list):
    """Outputs a dictionary of input file paths for a given list of input file paths."""
    file_dict = {}
    for file in file_list:
        file_dict[file.split('/')[-1]] = file
    return file_dict

In [30]:
# Make filepaths for feature files for genes, effectors, TE, methylation and randomised files for methylation.
feature_files = [fn for fn in glob.iglob('%s/*' % DIRS['FEATURES'], recursive=True)]
feature_fn_dict = file_name_dict(feature_files)

In [31]:
# This is the new file with only gene annotations
pprint.pprint(feature_fn_dict)

{'5mC_hc_tombo_sorted.cutoff.0.00.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.00.bed',
 '5mC_hc_tombo_sorted.cutoff.0.10.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.10.bed',
 '5mC_hc_tombo_sorted.cutoff.0.20.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.20.bed',
 '5mC_hc_tombo_sorted.cutoff.0.30.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.30.bed',
 '5mC_hc_tombo_sorted.cutoff.0.40.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.40.bed',
 '5mC_hc_tombo_sorted.cutoff.0.50.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.50.bed',
 '5mC_hc_tombo_sorted.cutoff.0.60.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.60.bed',
 '5mC_hc_tombo_sorted.cutoff.0.70.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.70.bed',
 '5mC_hc

In [853]:
# This is the original file
# Check whether the feature file dictionary works (it does). len = 59
pprint.pprint(feature_fn_dict)

{'5mC_hc_tombo_sorted.cutoff.0.00.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.00.bed',
 '5mC_hc_tombo_sorted.cutoff.0.10.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.10.bed',
 '5mC_hc_tombo_sorted.cutoff.0.20.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.20.bed',
 '5mC_hc_tombo_sorted.cutoff.0.30.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.30.bed',
 '5mC_hc_tombo_sorted.cutoff.0.40.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.40.bed',
 '5mC_hc_tombo_sorted.cutoff.0.50.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.50.bed',
 '5mC_hc_tombo_sorted.cutoff.0.60.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.60.bed',
 '5mC_hc_tombo_sorted.cutoff.0.70.bed': '/home/anjuni/analysis/coverage/feature_files/5mC_hc_tombo_sorted.cutoff.0.70.bed',
 '5mC_hc

In [ ]:
# Make dictionary of bedtools objects and check if it worked (it did!)
feature_bed_dict = make_bed_dict(feature_fn_dict)

In [ ]:
pprint.pprint(feature_bed_dict)

In [867]:
rand_files = [fn for fn in glob.iglob('%s/*' % DIRS['RAND'], recursive=True)]
rand_fn_dict = file_name_dict(rand_files)

In [869]:
# Check whether the feature file dictionary works (it does). len = 59
pprint.pprint(rand_fn_dict)

{'5mC_hc_tombo_sorted.cutoff.0.00_rand.bed': '/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.00_rand.bed',
 '5mC_hc_tombo_sorted.cutoff.0.10_rand.bed': '/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.10_rand.bed',
 '5mC_hc_tombo_sorted.cutoff.0.20_rand.bed': '/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.20_rand.bed',
 '5mC_hc_tombo_sorted.cutoff.0.30_rand.bed': '/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.30_rand.bed',
 '5mC_hc_tombo_sorted.cutoff.0.40_rand.bed': '/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.40_rand.bed',
 '5mC_hc_tombo_sorted.cutoff.0.50_rand.bed': '/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.50_rand.bed',
 '5mC_hc_tombo_sorted.cutoff.0.60_rand.bed': '/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.60_rand.bed',
 '5mC_hc_tombo_sorted.cutoff.0.70_rand.bed': '/home/anjuni/ana

In [870]:
# Make dictionary of bedtools objects and check if it worked (it did!)
rand_bed_dict = make_bed_dict(rand_fn_dict)

In [871]:
pprint.pprint(rand_bed_dict)

{'5mC_hc_tombo_sorted.cutoff.0.00_rand.bed': <BedTool(/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.00_rand.bed)>,
 '5mC_hc_tombo_sorted.cutoff.0.10_rand.bed': <BedTool(/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.10_rand.bed)>,
 '5mC_hc_tombo_sorted.cutoff.0.20_rand.bed': <BedTool(/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.20_rand.bed)>,
 '5mC_hc_tombo_sorted.cutoff.0.30_rand.bed': <BedTool(/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.30_rand.bed)>,
 '5mC_hc_tombo_sorted.cutoff.0.40_rand.bed': <BedTool(/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.40_rand.bed)>,
 '5mC_hc_tombo_sorted.cutoff.0.50_rand.bed': <BedTool(/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.50_rand.bed)>,
 '5mC_hc_tombo_sorted.cutoff.0.60_rand.bed': <BedTool(/home/anjuni/analysis/coverage/randomisation/5mC_hc_tombo_sorted.cutoff.0.60_rand.bed)>,

In [1417]:
# Make filepaths for feature files for TE superfamilies.
sf_files = [fn for fn in glob.iglob('%s/*' % DIRS['TE_SF'], recursive=True)]
sf_fn_dict = file_name_dict(sf_files)

In [209]:
DIRS['TE_SF']

'/home/anjuni/analysis/coverage/superfamily_files'

In [1420]:
# Make dictionary of bedtools objects and check if it worked (it did!)
sf_bed_dict = make_bed_dict(sf_fn_dict)

### <span style='color:red'> BUSCO and repeat gene and combined file <span/>

In [70]:
# make busco_bed_dict which includes the TE (LTgene) and combined file
feature_files_2 = [os.path.join(DIRS['FEATURES'], 'Pst_104E_v12_ph_busco.gene.bed'), \
                  os.path.join(DIRS['FEATURES'], 'Pst_104E_v12_ph_non_busco.gene.bed'),\
                  os.path.join(DIRS['FEATURES'], 'Pst_104E_v13_ph_ctg.TE.sorted.genes_only.gff3'),\
                  os.path.join(DIRS['FEATURES'], 'Pst_104E_v13_ph_ctg_combined_sorted_anno.genes_only.gff3'),]

feature_fn_dict2 = file_name_dict(feature_files_2)

In [71]:
feature_fn_dict2

{'Pst_104E_v12_ph_busco.gene.bed': '/home/anjuni/analysis/coverage/feature_files/Pst_104E_v12_ph_busco.gene.bed',
 'Pst_104E_v12_ph_non_busco.gene.bed': '/home/anjuni/analysis/coverage/feature_files/Pst_104E_v12_ph_non_busco.gene.bed',
 'Pst_104E_v13_ph_ctg.TE.sorted.genes_only.gff3': '/home/anjuni/analysis/coverage/feature_files/Pst_104E_v13_ph_ctg.TE.sorted.genes_only.gff3',
 'Pst_104E_v13_ph_ctg_combined_sorted_anno.genes_only.gff3': '/home/anjuni/analysis/coverage/feature_files/Pst_104E_v13_ph_ctg_combined_sorted_anno.genes_only.gff3'}

In [72]:
feature_bed_dict2 = make_bed_dict(feature_fn_dict2)

In [73]:
feature_bed_dict2

{'Pst_104E_v12_ph_busco.gene.bed': <BedTool(/home/anjuni/analysis/coverage/feature_files/Pst_104E_v12_ph_busco.gene.bed)>,
 'Pst_104E_v12_ph_non_busco.gene.bed': <BedTool(/home/anjuni/analysis/coverage/feature_files/Pst_104E_v12_ph_non_busco.gene.bed)>,
 'Pst_104E_v13_ph_ctg.TE.sorted.genes_only.gff3': <BedTool(/home/anjuni/analysis/coverage/feature_files/Pst_104E_v13_ph_ctg.TE.sorted.genes_only.gff3)>,
 'Pst_104E_v13_ph_ctg_combined_sorted_anno.genes_only.gff3': <BedTool(/home/anjuni/analysis/coverage/feature_files/Pst_104E_v13_ph_ctg_combined_sorted_anno.genes_only.gff3)>}

### <span style='color:red'> Transposon files. <span/>

In [213]:
# make list -> dict -> bed

te_fn_list = [fn for fn in glob.iglob('%s/*ph*REPET*' % DIRS['FEATURES'], recursive=True)]
te_fn_dict = file_name_dict(te_fn_list)
te_bed_dict = make_bed_dict(te_fn_dict)

### <span style='color:#148aff'> 5.C Using Ben's pybedtools coverage function on the feature files. <span/>

In [353]:
%%bash

# move the feature files to the correct folder
cd /home/anjuni/analysis/coverage
cp ../bedtools_output/intersects_from_cutoffs/* feature_files

In [75]:
# Make a function to do overlaps for feature files
def coverage(windowbed_dict, featurebed_dict, featurefn_dict, old_folder_name, new_folder_name):
    """Create coverage files from a dictionary of window pybedtools objects and a dictionary of feature file pybedtools objects. Also outputs a dictionary of pandas dataframes for all coverage files."""
    feature_overlap_df_dict = {}
    for wkey, wbed in windowbed_dict.items():
        for fkey, fbed in featurebed_dict.items():
            tmp_df = wbed.coverage(fbed).to_dataframe().iloc[:,[0,1,2,3,6]] # make a dataframe to put headings
            tmp_df.rename(columns={'name': 'overlap_count', 'thickStart': 'overlap_fraction'}, inplace=True) # rename headings
            if featurefn_dict[fkey].endswith('.bed'): # for methylation files
                tmp_fn = featurefn_dict[fkey].replace('.bed', '.%s.overlap.bed' % wkey) # change output file path
            if featurefn_dict[fkey].endswith('.gff3'): # for gene/transposon/effector/exon files
                tmp_fn = featurefn_dict[fkey].replace('.gff3', '.%s.overlap.bed' % wkey) # change output file path
            tmp_fn = tmp_fn.replace(old_folder_name, new_folder_name)
            feature_overlap_df_dict[tmp_fn.split('/')[-1]] = tmp_df # file name as key and dataframe as value for overlap dict
            tmp_df.to_csv(tmp_fn, sep='\t', header=None, index=None) # save to a csv(pybedtools outputs more d.p. than BEDTools)
    return feature_overlap_df_dict

In [ ]:
# Run overlaps between windows and features
feature_coverage_dict = coverage(window_bed_dict, feature_bed_dict, feature_fn_dict, 'feature_files', 'coverage_feature')
rand_coverage_dict = coverage(window_bed_dict, rand_bed_dict, rand_fn_dict, 'randomisation', 'coverage_random')

In [1438]:
# Run overlaps between windows and TE superfamily files
sf_coverage_dict = coverage(window_bed_dict, sf_bed_dict, sf_fn_dict, 'superfamily_files', 'coverage_superfamily')

In [76]:
# Run overlaps for the combined gene files, LTgene and BUSCO for consistency with the per-gene analysis
feature_coverage_dict2 = coverage(window_bed_dict, feature_bed_dict2, feature_fn_dict2, 'feature_files', 'coverage_feature')

In [69]:
window_bed_dict

{'10kb': <BedTool(/home/anjuni/analysis/windows/Pst_104E_v13_ph_ctg_w10kb.bed)>,
 '10kb_s2kb': <BedTool(/home/anjuni/analysis/windows/Pst_104E_v13_ph_ctg_w10kb_s2kb.bed)>}

In [215]:
te_coverage_dict = coverage(window_bed_dict, te_bed_dict, te_fn_dict, 'feature_files', 'coverage_feature')

### <span style='color:#148aff'> 5.D Statistical tests on the coverage files. <span/>

In [958]:
def separate_sliding_windows(input_dict, string):
    """Separates sliding and non-sliding windows into separate dictionaries."""
    sliding_dict = {}
    non_sliding_dict = {}
    for key, value in input_dict.items():
        if key.endswith(string):
            sliding_dict[key] = value
        else:
            non_sliding_dict[key] = value
    return sliding_dict, non_sliding_dict

In [959]:
rand_s_dict, rand_ns_dict = separate_sliding_windows(rand_coverage_dict, '_s2kb.overlap.bed')
feature_s_dict, feature_ns_dict = separate_sliding_windows(feature_coverage_dict, '_s2kb.overlap.bed')

In [1227]:
def coverage_wilcoxon(obs_df_dict, exp_df_dict, obs_slice, exp_slice):
    """This function returns a dictionary of chi square statistic and p-value for a test of observed and randomised sites."""
    wilcoxon_dict = {}
    for okey, ovalue in obs_df_dict.items():
        for ekey, evalue in exp_df_dict.items():
            if okey[:obs_slice] == ekey[:exp_slice]:
                o_df = ovalue
                e_df = evalue
                obs = o_df['overlap_fraction']
                exp = e_df['overlap_fraction']
                stat, p = wilcoxon(obs, exp)
                wilcoxon_dict[okey] = stat, p
    return wilcoxon_dict

In [1228]:
wilcoxon_coverage = coverage_wilcoxon(feature_ns_dict, rand_ns_dict, -17, -22)

/home/anjuni/anaconda3/lib/python3.6/site-packages/scipy/stats/morestats.py:2385: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
/home/anjuni/anaconda3/lib/python3.6/site-packages/scipy/stats/morestats.py:2410: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se


In [1233]:
pprint.pprint(wilcoxon_coverage)

{'5mC_hc_tombo_sorted.cutoff.0.00.10kb.overlap.bed': (87915.5, 0.0),
 '5mC_hc_tombo_sorted.cutoff.0.10.10kb.overlap.bed': (85739.5, 0.0),
 '5mC_hc_tombo_sorted.cutoff.0.20.10kb.overlap.bed': (101277.5, 0.0),
 '5mC_hc_tombo_sorted.cutoff.0.30.10kb.overlap.bed': (234278.0, 0.0),
 '5mC_hc_tombo_sorted.cutoff.0.40.10kb.overlap.bed': (2327553.5, 0.0),
 '5mC_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed': (16607427.5, 0.0),
 '5mC_hc_tombo_sorted.cutoff.0.60.10kb.overlap.bed': (46236315.0,
                                                      9.9306376916492e-187),
 '5mC_hc_tombo_sorted.cutoff.0.70.10kb.overlap.bed': (60573267.0,
                                                      2.6349063610233442e-05),
 '5mC_hc_tombo_sorted.cutoff.0.80.10kb.overlap.bed': (45245491.5,
                                                      1.2543957014880141e-206),
 '5mC_hc_tombo_sorted.cutoff.0.90.10kb.overlap.bed': (28260071.5, 0.0),
 '5mC_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed': (22742993.5, 0.0),
 

All the cutoffs had statistically significant results, so use any of them I guess. The lower cutoff would be best because you get more sites.

### <span style='color:#148aff'> 5.E Correlations between the coverage files. <span/>

In [77]:
#BUSCO, TE, combined
# 5mC correlation with genes
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v13_ph_ctg_combined_sorted_anno.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.276191145277843, pvalue=4.373185867943869e-278)

In [78]:
#BUSCO, TE, combined
# 5mC correlation with genes
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v13_ph_ctg_combined_sorted_anno.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.05686964165497747, pvalue=6.035942222516243e-13)

In [79]:
#BUSCO, TE, combined
# 5mC correlation with BUCO
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v12_ph_busco.gene.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.22601298568774855, pvalue=1.7881115150210445e-184)

In [80]:
#BUSCO, TE, combined
# 5mC correlation with BUSCO
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v12_ph_busco.gene.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.041272162132019084, pvalue=1.7601743750229128e-07)

In [82]:
#BUSCO, TE, combined
# 5mC correlation with non-BUSCO
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v12_ph_non_busco.gene.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.3678591679140834, pvalue=0.0)

In [81]:
#BUSCO, TE, combined
# 5mC correlation with non-BUSCO
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v12_ph_non_busco.gene.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.013482012862318015, pvalue=0.08809894531377398)

In [90]:
#BUSCO, TE, combined
# 5mC correlation with LTgene
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v13_ph_ctg.TE.sorted.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.31193039073123413, pvalue=0.0)

In [89]:
#BUSCO, TE, combined
# 5mC correlation with LTgene
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v13_ph_ctg.TE.sorted.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.13412724307796334, pvalue=3.889050036840751e-65)

In [87]:
#BUSCO, TE, combined
# 6mA correlation with genes
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v13_ph_ctg_combined_sorted_anno.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.16161991387643665, pvalue=4.157877429661693e-94)

In [88]:
#BUSCO, TE, combined
# 6mA correlation with genes
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v13_ph_ctg_combined_sorted_anno.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.016074770591923568, pvalue=0.0419977742242739)

In [86]:
#BUSCO, TE, combined
# 6ma correlation with BUSCO
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v12_ph_busco.gene.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.13466340766632134, pvalue=1.197121781530567e-65)

In [85]:
#BUSCO, TE, combined
# 6ma correlation with BUSCO
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v12_ph_busco.gene.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.004037043082224927, pvalue=0.6095779046477805)

In [84]:
# 6mA correlation with non-BUSCO
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v12_ph_non_busco.gene.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.20982437331850182, pvalue=1.0605283805660893e-158)

In [83]:
# 6mA correlation with non-BUSCO
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v12_ph_non_busco.gene.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.07711638886693978, pvalue=1.5213427712988084e-22)

In [92]:
# 6mA correlation with LTgene
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v13_ph_ctg.TE.sorted.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.16052992708770275, pvalue=7.489128681731838e-93)

In [91]:
# 6mA correlation with LTgene
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict2['Pst_104E_v13_ph_ctg.TE.sorted.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.09464632091001793, pvalue=3.5778541705521973e-33)

In [36]:
#remaking feature coverage dict
feature_list = [fn for fn in glob.iglob('%s/*.10kb.overlap.bed' % os.path.join(DIRS['COVERAGE'], 'coverage_feature'), recursive=True)]


In [37]:
headings_feature = ['contig', 'start', 'stop', 'overlap_count', 'overlap_fraction']

In [40]:
feature_coverage_dict = {}
for file in feature_list:
    df = pd.read_csv(file, sep='\t', header = None, names = headings_feature)
    feature_coverage_dict[file.split('/')[-1]] = df

In [42]:
# 5mC correlation with genes
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.anno.sorted.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.44154782319095753, pvalue=0.0)

In [43]:
# 5mC correlation with genes
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.anno.sorted.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.028409315871959166, pvalue=0.0003251154331316708)

In [41]:
# 5mC correlation with TEs
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.TE.sorted.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.31193039073123413, pvalue=0.0)

In [45]:
# 5mC correlation with TEs
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.TE.sorted.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.13412724307796334, pvalue=3.889050036840751e-65)

In [44]:
# 5mC correlation with TEs
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.TE.sorted.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.31193039073123413, pvalue=0.0)

In [48]:
# 5mC correlation with effectors
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.effectors.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.021787117952869847, pvalue=0.005845345788267107)

In [51]:
# 5mC correlation with effectors
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.effectors.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.1525946319854954, pvalue=5.589471803253034e-84)

In [50]:
# 6mA correlation with genes
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.anno.sorted.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.07374523463007764, pvalue=9.518171150855629e-21)

In [52]:
# 6mA correlation with genes
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.anno.sorted.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.251677503930484, pvalue=1.0475308033426758e-229)

In [53]:
# 6mA correlation with TEs
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.50.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.TE.sorted.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.09464632091001793, pvalue=3.5778541705521973e-33)

In [54]:
# 6mA correlation with TEs
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.95.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.TE.sorted.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.16052992708770275, pvalue=7.489128681731838e-93)

In [1243]:
# 5mC correlation with genes
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.99.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.anno.sorted.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.38281643716088243, pvalue=0.0)

In [1244]:
# 5mC correlation with TEs
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.99.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.TE.sorted.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.25369718437406363, pvalue=1.690789556321972e-233)

In [1245]:
# 5mC correlation with effectors
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.99.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.effectors.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.13931099497837787, pvalue=3.5837515997418266e-70)

In [1250]:
# 6mA correlation with genes
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.99.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.anno.sorted.genes_only.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.18353279200007624, pvalue=2.970389253688614e-121)

In [1264]:
# 6mA correlation with effectors
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.99.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.effectors.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.07515396006471368, pvalue=1.728156969082245e-21)

In [1262]:
# 6mA correlation with TEs
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.99.10kb.overlap.bed'].overlap_fraction,\
         feature_coverage_dict['Pst_104E_v13_ph_ctg.TE.sorted.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.09330526525126864, pvalue=2.775701706245244e-32)

In [ ]:
# Test correlation between randomised files and feature files as a control

In [1440]:
# Test correlation between 5mC and superfamily files
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.99.10kb.overlap.bed'].overlap_fraction,\
         sf_coverage_dict['Pst_104E_v13_ph_ctg.ClassI:LTR:Gypsy.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.06080346779741759, pvalue=1.3758476935372227e-14)

In [1441]:
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.99.10kb.overlap.bed'].overlap_fraction,\
         sf_coverage_dict['Pst_104E_v13_ph_ctg.ClassI:LTR:Gypsy.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.017815652590927895, pvalue=0.02420836921035543)

In [1442]:
spearmanr(feature_coverage_dict['5mC_hc_tombo_sorted.cutoff.0.99.10kb.overlap.bed'].overlap_fraction,\
         sf_coverage_dict['Pst_104E_v13_ph_ctg.ClassI:LTR:Copia.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=0.0268650466727745, pvalue=0.0006764206901908494)

In [1444]:
spearmanr(feature_coverage_dict['6mA_hc_tombo_sorted.cutoff.0.99.10kb.overlap.bed'].overlap_fraction,\
         sf_coverage_dict['Pst_104E_v13_ph_ctg.ClassI:LTR:Copia.10kb.overlap.bed'].overlap_fraction)

SpearmanrResult(correlation=-0.02353519583145995, pvalue=0.0029056708810170496)

In [ ]:
# Superfamily spearman

In [ ]:
def spearman_windows():

In [2599]:
# Make a function to do overlaps for feature files
def spearman_superfamily(windowbed_dict, featurebed_dict):
    """Runs spearman."""
    spearman_overlap_df_dict = {}
    for wkey, wvalue in windowbed_dict.items():
            if wkey.endswith('.10kb.overlap.bed'):
                for fkey, fvalue in featurebed_dict.items():
                        if fkey.endswith('_sorted.cutoff.0.10.10kb.overlap.bed'):
                            o_df = wvalue
                            e_df = fvalue
                            obs = o_df['overlap_fraction']
                            exp = e_df['overlap_fraction']
                            stat, p = spearmanr(obs, exp)
                            skey = fkey[:4] + wkey
                            spearman_overlap_df_dict[skey] = stat, p
    return spearman_overlap_df_dict

In [2587]:
sf_spearman_dict = spearman_superfamily(sf_coverage_dict, feature_coverage_dict)

In [2588]:
sf_spearman_dict

{'5mC_Pst_104E_v13_ph_ctg.ClassI:?:?.10kb.overlap.bed': (-0.005052195920238353,
  0.5227631585879537),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:DIRS:?.10kb.overlap.bed': (-0.004370705001921898,
  0.5803429355391925),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:DIRS:DIRS.10kb.overlap.bed': (0.008906687305393228,
  0.25987277407033427),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LARD:?.10kb.overlap.bed': (0.03251363204764782,
  3.8891930614113356e-05),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LINE:?.10kb.overlap.bed': (0.005817613465915172,
  0.46178115951908294),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LINE:I.10kb.overlap.bed': (0.007221378136947359,
  0.3609823479805958),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LINE:Jockey.10kb.overlap.bed': (0.015330423963023887,
  0.05245664853297324),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LINE:L1.10kb.overlap.bed': (-0.0010651437863885488,
  0.8928193677382682),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LINE:R2.10kb.overlap.bed': (0.008084198315614221,
  0.306476277824157),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LINE:R

In [2595]:
# changed line 8 to (( if fkey.endswith('_sorted.cutoff.0.10.10kb.overlap.bed'): ))
sf_99_spearman_dict = spearman_superfamily(sf_coverage_dict, feature_coverage_dict)

In [2596]:
sf_99_spearman_dict

{'5mC_Pst_104E_v13_ph_ctg.ClassI:?:?.10kb.overlap.bed': (-0.006682304870588272,
  0.39794269491180734),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:DIRS:?.10kb.overlap.bed': (0.016606868083582138,
  0.03565286745555298),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:DIRS:DIRS.10kb.overlap.bed': (-0.00042559678567772376,
  0.9570652423066557),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LARD:?.10kb.overlap.bed': (-0.05266453242100205,
  2.6191593340653004e-11),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LINE:?.10kb.overlap.bed': (0.009290460732737833,
  0.2398982706039067),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LINE:I.10kb.overlap.bed': (0.005087523214012956,
  0.5198602965466651),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LINE:Jockey.10kb.overlap.bed': (0.025769382452240995,
  0.0011129762400651705),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LINE:L1.10kb.overlap.bed': (0.020563799316138866,
  0.009280767119246138),
 '5mC_Pst_104E_v13_ph_ctg.ClassI:LINE:R2.10kb.overlap.bed': (-0.003409230176774914,
  0.6662802299800441),
 '5mC_Pst_104E_v13_ph_ctg.ClassI

Overall, very weak correlations between methylation and key features in the genome, especially TE superfamilies :)